In [58]:
# pandas 라이브러리를 pd로 임포트
import pandas as pd

# 파일로부터 정보를 추출하는 함수 정의
def extract_information(file_path):
    try:
        # 파일을 읽어와서 문헌 정보를 \n\n으로 나누어 리스트로 저장
        with open(file_path, 'r', encoding='utf-8') as file:
            documents = file.read().split('\n\n')  # 각 문헌은 \n\n으로 구분되어 있다고 가정
    except FileNotFoundError:
        # 파일을 찾을 수 없는 경우 에러 메시지 출력하고 빈 리스트 반환
        print(f"Error: 파일 '{file_path}'을 찾을 수 없습니다.")
        return []
    except Exception as e:
        # 파일 읽기 중 오류 발생 시 에러 메시지 출력하고 빈 리스트 반환
        print(f"Error: 파일을 읽는 중 오류가 발생했습니다. {e}")
        return []

    # 추출된 정보를 담을 빈 리스트
    extracted_data = []

    # 문헌 정보를 순회하며 처리
    for document in documents:
        # 문헌 정보를 줄 단위로 나눠 리스트로 저장
        lines = document.split('\n')

        # 문헌의 최소한 2줄이어야 함을 확인하고 그렇지 않으면 스킵
        if len(lines) < 2:
            print(f"Warning: 문헌 형식이 올바르지 않습니다. 스킵합니다.")
            continue

        # 문헌의 첫 번째 줄(저자 정보) 삭제
        document = document.split('\n', 1)[0]

        # 각 문헌의 제목, 연도, 키워드, 초록 추출
        title, year, keywords, abstract = lines[1], lines[4], "", ""

        # Abstract이라는 단어가 없다면 공백으로 비우고 다음으로 넘어가기
        if "Abstract" not in lines:
            abstract = ""

        # Abstract이라는 단어 뒤부터 Keywords가 나오기 전까지 추출
        abstract_flag = False
        newline_count = 0
        for line in lines[3:]:
            if "Abstract" in line:
                abstract_flag = True
            if abstract_flag:
                abstract += line + "\n"
                if not line.strip():
                    newline_count += 1
                # Keywords가 나오면 루프 종료
                if (newline_count == 1) and ("Keywords" in line):
                    break
                # Abstract이 두 번째 줄까지 나오면 루프 종료
                elif newline_count == 2:
                    break

        # Keywords라는 단어가 없다면 공백으로 비우고 다음으로 넘어가기
        if "Keywords" not in lines:
            keywords = ""

        # Keywords라는 단어 뒤부터 \n(줄바꿈)이 두 번 나오기 전까지 추출
        keywords_flag = False
        newline_count = 0
        for line in lines[3:]:
            if "Keywords" in line:
                keywords_flag = True
            if keywords_flag:
                keywords += line + "\n"
                if not line.strip():
                    newline_count += 1
                # \n(줄바꿈)이 두 번 나오면 루프 종료
                if newline_count == 2:
                    break

        # 추출한 정보를 리스트에 추가
        extracted_data.append({
            'Title': title,
            'Year': year,
            'Keywords': keywords.replace("Keywords:", "").strip(),  # "Keywords:" 삭제 후 좌우 공백 제거
            'Abstract': abstract.replace("Abstract:", "").strip()  # "Abstract:" 삭제 후 좌우 공백 제거
        })

    # 완성된 문헌 정보 리스트 반환
    return extracted_data

# 정보 담은 파일 경로 설정
file_path = 'total.txt'
# 함수를 통해 정보 추출
result = extract_information(file_path)

# 추출된 정보를 데이터프레임으로 변환
df = pd.DataFrame(result)

# 데이터프레임 출력
print(df)

                                                   Title             Year  \
0      Somali refugee women speak out about their nee...            2004,   
1      Treatment of pharmacotherapy-refractory posttr...            2003,   
2      Development of an inventory for measuring war-...            2005,   
3      Women, international migration and self-report...            2003,   
4      On the battlefield of women's bodies: An overv...            2004,   
...                                                  ...              ...   
14720  Immigration, ethnic diversity and voting: The ...            2020,   
14721  Pioneers in CNS inhibition: 2. Charles Sherrin...            2020,   
14722                                          Contents,            2020,   
14723  Impact of COVID-19 pandemic on the Jordanian e...            2022,   
14724                          Chapter 1 - Introduction,  Academic Press,   

                                                Keywords  \
0      Somalia;

In [72]:
import re  # 정규 표현식을 사용하기 위해 re 라이브러리 임포트

# pandas 라이브러리를 pd로 임포트
import pandas as pd

# 파일로부터 정보를 추출하는 함수 정의
def extract_information(file_path):
    try:
        # 파일을 읽어와서 문헌 정보를 \n\n으로 나누어 리스트로 저장
        with open(file_path, 'r', encoding='utf-8') as file:
            documents = file.read().split('\n\n')  # 각 문헌은 \n\n으로 구분되어 있다고 가정
    except FileNotFoundError:
        # 파일을 찾을 수 없는 경우 에러 메시지 출력하고 빈 리스트 반환
        print(f"Error: 파일 '{file_path}'을 찾을 수 없습니다.")
        return []
    except Exception as e:
        # 파일 읽기 중 오류 발생 시 에러 메시지 출력하고 빈 리스트 반환
        print(f"Error: 파일을 읽는 중 오류가 발생했습니다. {e}")
        return []

    # 추출된 정보를 담을 빈 리스트
    extracted_data = []

    # 문헌 정보를 순회하며 처리
    for document in documents:
        # 문헌 정보를 줄 단위로 나눠 리스트로 저장
        lines = document.split('\n')

        # 문헌의 최소한 2줄이어야 함을 확인하고 그렇지 않으면 스킵
        if len(lines) < 2:
            print(f"Warning: 문헌 형식이 올바르지 않습니다. 스킵합니다.")
            continue

        # 문헌의 첫 번째 줄(저자 정보) 삭제
        document = document.split('\n', 1)[0]

        # 각 문헌의 제목, 연도, 키워드, 초록 추출
        title, year, keywords, abstract = lines[1], "", "", ""

        # Abstract이라는 단어가 없다면 공백으로 비우고 다음으로 넘어가기
        if "Abstract" not in lines:
            abstract = ""

        # Abstract이라는 단어 뒤부터 Keywords가 나오기 전까지 추출
        abstract_flag = False
        newline_count = 0
        for line in lines[3:]:
            if "Abstract" in line:
                abstract_flag = True
            if abstract_flag:
                abstract += line + "\n"
                if not line.strip():
                    newline_count += 1
                # Keywords가 나오면 루프 종료
                if (newline_count == 1) and ("Keywords" in line):
                    break
                # Abstract이 두 번째 줄까지 나오면 루프 종료
                elif newline_count == 2:
                    break

        # Keywords라는 단어가 없다면 공백으로 비우고 다음으로 넘어가기
        if "Keywords" not in lines:
            keywords = ""

        # Keywords라는 단어 뒤부터 \n(줄바꿈)이 두 번 나오기 전까지 추출
        keywords_flag = False
        newline_count = 0
        for line in lines[3:]:
            if "Keywords" in line:
                keywords_flag = True
            if keywords_flag:
                keywords += line + "\n"
                if not line.strip():
                    newline_count += 1
                # \n(줄바꿈)이 두 번 나오면 루프 종료
                if newline_count == 2:
                    break

        # 년도 추출을 위한 정규 표현식(\b\d{4}\b : 4자리 숫자로 이루어진 단어를 찾는 정규 표현식) 사용
        year_match = re.search(r'\b\d{4}\b', lines[4])
        # 정규 표현식으로 추출한 결과가 있다면 해당 결과를 year로 사용, 없다면 다음 라인에서 추출
        if year_match:
            year = year_match.group()
        elif True:
            year_match = re.search(r'\b\d{4}\b', lines[5])
            # 정규 표현식으로 추출한 결과가 있다면 해당 결과를 year로 사용, 없다면 빈 문자열
            year = year_match.group() if year_match else ""
        else:
            year_match = re.search(r'\b\d{4}\b', lines[6])
            # 정규 표현식으로 추출한 결과가 있다면 해당 결과를 year로 사용, 없다면 빈 문자열
            year = year_match.group() if year_match else ""

        # 추출한 정보를 리스트에 추가
        extracted_data.append({
            'Title': title,
            'Year': year,
            'Keywords': keywords.replace("Keywords:", "").strip(),  # "Keywords:" 삭제 후 좌우 공백 제거
            'Abstract': abstract.replace("Abstract:", "").strip()  # "Abstract:" 삭제 후 좌우 공백 제거
        })

    # 완성된 문헌 정보 리스트 반환
    return extracted_data

# 정보 담은 파일 경로 설정
file_path = 'total.txt'
# 함수를 통해 정보 추출
result = extract_information(file_path)

# 추출된 정보를 데이터프레임으로 변환
df = pd.DataFrame(result)

# 데이터프레임 출력
print(df)

                                                   Title  Year  \
0      Somali refugee women speak out about their nee...  2004   
1      Treatment of pharmacotherapy-refractory posttr...  2003   
2      Development of an inventory for measuring war-...  2005   
3      Women, international migration and self-report...  2003   
4      On the battlefield of women's bodies: An overv...  2004   
...                                                  ...   ...   
14720  Immigration, ethnic diversity and voting: The ...  2020   
14721  Pioneers in CNS inhibition: 2. Charles Sherrin...  2020   
14722                                          Contents,  2020   
14723  Impact of COVID-19 pandemic on the Jordanian e...  2022   
14724                          Chapter 1 - Introduction,  2023   

                                                Keywords  \
0      Somalia; refugees; childbirth; labor; delivery...   
1      Posttraumatic stress disorder; Refugee; Cambod...   
2                          

In [73]:
# 다운할 파일 경로 설정
excel_file_path = 'sorted_total.xlsx'

# 데이터프레임을 엑셀 파일로 저장
df.to_excel(excel_file_path, index=False)

print(f"엑셀 파일이 생성되었습니다: {excel_file_path}")

엑셀 파일이 생성되었습니다: sorted_total.xlsx
